CA2 Programming for data analytics

Student Name:Manoj Sattu Kamble - 10600303

Div: A

**Extract, Transform & Load**

In [ ]:
import pandas as pd
import re
!pip install pypyodbc
import pypyodbc
import numpy as np
from datetime import datetime, timedelta

In [ ]:
#Load the CSV file (using Python).

df = pd.read_csv("/content/FireBrigadeAndAmbulanceCallOuts.csv")

#Output the total number of rows and columns.
rows = df.shape[0]
cols = df.shape[1]
print("Rows: " + str(rows))
print("Columns: " + str(cols))

Rows: 38556
Columns: 11


**Output the number of non-null rows (by column)**

In [ ]:
df.notnull().sum()

Date            38556
Station Area    38556
Description     38556
TOC             38556
ORD             38556
MOB             37311
IA              30962
LS              38556
AH                153
MAV             38530
CD              38530
dtype: int64

**Output the number of null values (by column).**


In [ ]:
df.isnull().sum()

Date                0
Station Area        0
Description         0
TOC                 0
ORD                 0
MOB              1245
IA               7594
LS                  0
AH              38403
MAV                26
CD                 26
dtype: int64

**Output the number of null values for all columns.**

In [ ]:
df.isnull().sum().sum()

47294

**Output the total number of call outs by Station Area.**


In [ ]:
df = df.groupby(['Station Area'])['Station Area'].count()
print(df)

Station Area
Balbriggan         688
Blanchardstown    2089
Dolphins Barn     4018
Donnybrook        1787
Dun Laoghaire     3271
Dunshaughlin         2
Finglas           3030
Kilbarrack        3548
MH14                 1
North Strand      1645
Phibsborough      3160
Rathfarnham       2129
Skerries           449
Swords            2614
Tallaght          6525
Tara St           3600
Name: Station Area, dtype: int64


**Output the total number of call outs by Station Area and Date where the description is either Fire Car or Fire Alarm.**

In [ ]:
#Step 1 grouping the data by Station area and Date
df_2 = pd.read_csv("/content/FireBrigadeAndAmbulanceCallOuts.csv")
df3 = df_2.groupby(['Station Area', 'Date']).size().reset_index(name='count')
print(df3)

      Station Area       Date  count
0       Balbriggan   1/1/2013      3
1       Balbriggan   1/1/2014      1
2       Balbriggan  1/10/2015      1
3       Balbriggan  1/11/2014      1
4       Balbriggan  1/11/2015      4
...            ...        ...    ...
12478      Tara St   9/8/2014      6
12479      Tara St   9/8/2015      4
12480      Tara St   9/9/2013      2
12481      Tara St   9/9/2014      8
12482      Tara St   9/9/2015      1

[12483 rows x 3 columns]


In [ ]:
#Step 2 filtering data where description is either car or fire alarm
filt = df_2[(df_2.Description == "Fire CAR") | (df_2.Description == "Fire ALARM")]
filt.groupby(['Date','Station Area']).size()                                     

Date      Station Area  
1/1/2013  Balbriggan        3
          Blanchardstown    1
          Dun Laoghaire     1
          Kilbarrack        2
          Tallaght          4
                           ..
9/9/2015  Finglas           1
          Kilbarrack        1
          Swords            3
          Tallaght          4
          Tara St           1
Length: 6792, dtype: int64

**Replace any instance of “,” and "-" (in any column) with an empty string.**


In [ ]:
df4 = df_2.replace([',','-'],'')

**Drop rows for the columns (AH, MAV, CD) where at least one row value is NULL.**

In [ ]:
df5 = df4.dropna(subset=['AH', 'MAV', 'CD'])

**Drop any duplicate rows (except for the first occurrence)**

In [ ]:
df6 = df5.drop_duplicates()
df6

,Date,Station Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD
0,1/1/2013,Rathfarnham,S/S OTHER,1:27:19,1:28:58,1:30:47,1:58:54,2:08:54,2:28:54,2:48:54,3:08:54
1,1/1/2013,Tallaght,Fire CAR,1:49:57,1:50:40,1:51:47,1:55:43,2:05:43,2:25:43,2:45:43,3:05:43
2,1/1/2013,North Strand,S/S RTA,3:35:24,3:36:14,3:38:28,3:41:20,3:51:20,4:11:20,4:31:20,4:51:20
3,1/1/2013,Tallaght,Fire CAR,4:12:51,4:13:56,4:14:32,4:20:48,4:30:48,4:50:48,5:10:48,5:30:48
4,1/1/2013,Finglas,Fire DOM PER,4:14:24,4:15:36,4:17:37,4:20:47,4:30:47,4:50:47,5:10:47,5:30:47
...,...,...,...,...,...,...,...,...,...,...,...
38490,16/10/2015,Tallaght,Fire SMALL,23:48:59,23:51:14,23:52:31,0:17:39,0:27:39,,0:20:00,0:40:00
38491,13/08/2015,Tallaght,Fire GRASS,23:49:10,23:51:28,23:53:04,NaN,0:10:00,,0:20:00,0:40:00
38496,23/06/2015,Finglas,Fire SMALL,23:49:04,23:52:19,23:53:42,NaN,0:10:00,,0:20:00,0:40:00
38497,23/12/2015,Dun Laoghaire,S/S TREEDN,23:50:52,23:52:20,23:54:17,0:02:43,0:12:43,,0:20:00,0:40:00


**Output the minimum time difference between TOC and ORD**

In [ ]:
#Created function for formatting time HH:MM:SS
def formatTimedelta(td):
    
    total = td.total_seconds()
    prefix, total = ('-', total*-1) if total < 0 else ('', total)
    h, r = divmod(total, 3600)
    m, s = divmod(r, 60)
    return f"{prefix}{int(h):02d}:{int(m):02d}:{int(s):02d}"

#Created temporary columns to store formatted time
df6['TOC_Temp'] = pd.to_datetime(df6['TOC'])
df6['ORD_Temp'] = pd.to_datetime(df6['ORD'])

# calculate time difference using timedeltas
df6['Difference'] = (df6['ORD_Temp']-df6['TOC_Temp']).apply(formatTimedelta)
df6

#dropping temporary colums from data frames
newdf = df6.drop(['TOC_Temp', 'ORD_Temp'], axis=1)
newdf

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Date,Station Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD,Difference
0,1/1/2013,Rathfarnham,S/S OTHER,1:27:19,1:28:58,1:30:47,1:58:54,2:08:54,2:28:54,2:48:54,3:08:54,00:01:39
1,1/1/2013,Tallaght,Fire CAR,1:49:57,1:50:40,1:51:47,1:55:43,2:05:43,2:25:43,2:45:43,3:05:43,00:00:43
2,1/1/2013,North Strand,S/S RTA,3:35:24,3:36:14,3:38:28,3:41:20,3:51:20,4:11:20,4:31:20,4:51:20,00:00:50
3,1/1/2013,Tallaght,Fire CAR,4:12:51,4:13:56,4:14:32,4:20:48,4:30:48,4:50:48,5:10:48,5:30:48,00:01:05
4,1/1/2013,Finglas,Fire DOM PER,4:14:24,4:15:36,4:17:37,4:20:47,4:30:47,4:50:47,5:10:47,5:30:47,00:01:12
...,...,...,...,...,...,...,...,...,...,...,...,...
38490,16/10/2015,Tallaght,Fire SMALL,23:48:59,23:51:14,23:52:31,0:17:39,0:27:39,,0:20:00,0:40:00,00:02:15
38491,13/08/2015,Tallaght,Fire GRASS,23:49:10,23:51:28,23:53:04,NaN,0:10:00,,0:20:00,0:40:00,00:02:18
38496,23/06/2015,Finglas,Fire SMALL,23:49:04,23:52:19,23:53:42,NaN,0:10:00,,0:20:00,0:40:00,00:03:15
38497,23/12/2015,Dun Laoghaire,S/S TREEDN,23:50:52,23:52:20,23:54:17,0:02:43,0:12:43,,0:20:00,0:40:00,00:01:28


In [ ]:
#renamed this particular column so that we can push it to the Database table
newdf.columns = newdf.columns.str.replace('Station Area', 'Station_Area')
newdf

,Date,Station_Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD,Difference
0,1/1/2013,Rathfarnham,S/S OTHER,1:27:19,1:28:58,1:30:47,1:58:54,2:08:54,2:28:54,2:48:54,3:08:54,00:01:39
1,1/1/2013,Tallaght,Fire CAR,1:49:57,1:50:40,1:51:47,1:55:43,2:05:43,2:25:43,2:45:43,3:05:43,00:00:43
2,1/1/2013,North Strand,S/S RTA,3:35:24,3:36:14,3:38:28,3:41:20,3:51:20,4:11:20,4:31:20,4:51:20,00:00:50
3,1/1/2013,Tallaght,Fire CAR,4:12:51,4:13:56,4:14:32,4:20:48,4:30:48,4:50:48,5:10:48,5:30:48,00:01:05
4,1/1/2013,Finglas,Fire DOM PER,4:14:24,4:15:36,4:17:37,4:20:47,4:30:47,4:50:47,5:10:47,5:30:47,00:01:12
...,...,...,...,...,...,...,...,...,...,...,...,...
38490,16/10/2015,Tallaght,Fire SMALL,23:48:59,23:51:14,23:52:31,0:17:39,0:27:39,,0:20:00,0:40:00,00:02:15
38491,13/08/2015,Tallaght,Fire GRASS,23:49:10,23:51:28,23:53:04,NaN,0:10:00,,0:20:00,0:40:00,00:02:18
38496,23/06/2015,Finglas,Fire SMALL,23:49:04,23:52:19,23:53:42,NaN,0:10:00,,0:20:00,0:40:00,00:03:15
38497,23/12/2015,Dun Laoghaire,S/S TREEDN,23:50:52,23:52:20,23:54:17,0:02:43,0:12:43,,0:20:00,0:40:00,00:01:28


In [ ]:
#replacing NULL data as 'NA'
newdf.replace({np.inf: np.nan, -np.inf: np.nan}, inplace=True)
df7 = newdf.fillna(value="NA")
df7

,Date,Station_Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD,Difference
0,1/1/2013,Rathfarnham,S/S OTHER,1:27:19,1:28:58,1:30:47,1:58:54,2:08:54,2:28:54,2:48:54,3:08:54,00:01:39
1,1/1/2013,Tallaght,Fire CAR,1:49:57,1:50:40,1:51:47,1:55:43,2:05:43,2:25:43,2:45:43,3:05:43,00:00:43
2,1/1/2013,North Strand,S/S RTA,3:35:24,3:36:14,3:38:28,3:41:20,3:51:20,4:11:20,4:31:20,4:51:20,00:00:50
3,1/1/2013,Tallaght,Fire CAR,4:12:51,4:13:56,4:14:32,4:20:48,4:30:48,4:50:48,5:10:48,5:30:48,00:01:05
4,1/1/2013,Finglas,Fire DOM PER,4:14:24,4:15:36,4:17:37,4:20:47,4:30:47,4:50:47,5:10:47,5:30:47,00:01:12
...,...,...,...,...,...,...,...,...,...,...,...,...
38490,16/10/2015,Tallaght,Fire SMALL,23:48:59,23:51:14,23:52:31,0:17:39,0:27:39,,0:20:00,0:40:00,00:02:15
38491,13/08/2015,Tallaght,Fire GRASS,23:49:10,23:51:28,23:53:04,NA,0:10:00,,0:20:00,0:40:00,00:02:18
38496,23/06/2015,Finglas,Fire SMALL,23:49:04,23:52:19,23:53:42,NA,0:10:00,,0:20:00,0:40:00,00:03:15
38497,23/12/2015,Dun Laoghaire,S/S TREEDN,23:50:52,23:52:20,23:54:17,0:02:43,0:12:43,,0:20:00,0:40:00,00:01:28


In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [4,011 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 https://packages.microsoft.com/ubuntu/16.04/prod xenial/main amd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key output should not be parsed (stdout is not a terminal)
100   983  100   983    0     0   4388      0 --:--:-- --:--:-- --:--:--  4388
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    79  100    79    0     0    523      0 --:--:-- --:--:-- --:--:--   523
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling

In [ ]:
# Connect to SQL Server
connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
'Server=40.113.221.206;'
'Database=master;'
'encrypt=yes;'
'TrustServerCertificate=yes;'
'UID=sa;'
'PWD=Manoj#21',autocommit = True)

In [ ]:
cursor = connection.cursor()
SQLCommand = ("CREATE DATABASE ManojDB;")
cursor.execute(SQLCommand)
print('done')

connection.close()

done


In [ ]:
connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
'Server=40.113.221.206;'
'Database=ManojDB;'
'encrypt=yes;'
'TrustServerCertificate=yes;'
'UID=sa;'
'PWD=Manoj#21',autocommit = True)

cursor = connection.cursor()

cursor.execute('''
CREATE TABLE FireBrigadeAndAmbulanceCallOuts(
  Date varchar(255),
  Station_Area varchar(255),
  Description varchar(255),
  TOC varchar(255),
  ORD varchar(255),
  MOB varchar(255),
  IA varchar(255),
  LS varchar(255),
  AH varchar(255),
  MAV varchar(255),
  CD varchar(255),
  Difference varchar(255)
  )
  ''')

In [ ]:
# Inserting DataFrame to Table

for index,row in df7.iterrows():
  query = "INSERT INTO FireBrigadeAndAmbulanceCallOuts (Date,Station_Area,Description,TOC,ORD,MOB,IA,LS,AH,MAV,CD,Difference) VALUES (?,?,?,?,?,?,?,?,?,?,?,?)"
  cursor.execute(query,(row.Date, row.Station_Area, row.Description, row.TOC, row.ORD, row.MOB, row.IA, row.LS, row.AH, row.MAV, row.CD, row.Difference))

In [ ]:
connection.commit()

https://colab.research.google.com/drive/10mr1_8is5T6EKI6dgdiJJXuJeytp0KKv?usp=sharing